In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
from importlib import reload
import os
import numpy as np
import json

In [ ]:
avg_feat_path = 'C:\\Users\\mathiopv\\OneDrive - Charité - Universitätsmedizin Berlin\\BATTERY_LIFE\\results\\Avg_Features'

In [ ]:
file_path

In [ ]:
[filename.split('_')[0]]*df.shape[0]

In [ ]:
# Initialize an empty list to store the DataFrames
dfs = []

# Iterate through the files in the directory
for filename in os.listdir(avg_feat_path):
    if filename.endswith(".json"):
        # Construct the full file path
        file_path = os.path.join(avg_feat_path, filename)
        f = open(file_path)
        data1 = json.load(f)
        df = pd.read_json(data1, orient ='index')
        df.reset_index(inplace=True)
        df.rename(columns={'index': 'FollowUp'}, inplace=True)

        df['SubID'] = [filename.split('_')[0]]*df.shape[0]
        dfs.append(df)


All_AvgFeatures = pd.concat(dfs, ignore_index=True)

In [ ]:
np.unique(All_AvgFeatures['SubID'])

In [ ]:
# Filter out rows with FollowUp as 'Beelitz'
beelitz_rows = All_AvgFeatures[All_AvgFeatures['FollowUp'] == 'Beelitz']

# Group by 'SubID' and apply custom aggregation
agg_funcs = {
    'Telemetry Duration Sum': 'sum',
    'Sensing Duration Sum': 'sum',
    'Last Battery Value': 'first',  # Keep only the first value, which will be 'Beelitz'
}

result_df = beelitz_rows.groupby('SubID').agg(agg_funcs)

# Rename 'Beelitz' to 'FU0M' for the 'FollowUp' column in the result DataFrame
result_df.rename(index={'Beelitz': 'FU0M'}, inplace=True)

result_df